# Playground for MVPA

Testing and trying to set up MVPA analysis using data from own fMRI experiment.

In [52]:
from pathlib import Path
from tools import get_events
from nilearn import plotting, image
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
import scipy.io as sio
import numpy as np
import pylab
import nibabel as nb
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

output_dir = Path.cwd() / "results" / "plot_two_runs_model"
output_dir.mkdir(exist_ok=True, parents=True)
print(f"Output will be saved to: {output_dir}")

Output will be saved to: C:\Users\skarkosz\Desktop\test_nilearn\results\plot_two_runs_model


In [53]:
# Loading exemplary subject
img_run1 = nb.load("sub-B045/ses-1/func/sub-B045_ses-1_task-fmri_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz")
img_run2 = nb.load("sub-B045/ses-1/func/sub-B045_ses-1_task-fmri_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz")
fmri_imgs = [img_run1,img_run2]
t1 = nb.load("sub-B045/anat/sub-B045_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz")


In [54]:
# Preparing list of stimuli
events_1 = get_events("trigger/sub-B045/ses-1/sub-B045_ses-1_Run1.mat")
events_2 = get_events("trigger/sub-B045/ses-1/sub-B045_ses-1_Run2.mat")

In [55]:
fmri_glm = FirstLevelModel(t_r=2,
                standardize=False,
                hrf_model='spm',
                drift_model='cosine',
                high_pass=0.01)

In [56]:
# Fitting First Level model
fst_lvl = fmri_glm.fit(fmri_imgs,events=[events_1,events_2])

In [57]:
#Checking size of desing matrices
matrices = fst_lvl.design_matrices_
print(f"Matrices have shapes: {matrices[0].shape}, {matrices[1].shape}")

Matrices have shapes: (400, 19), (529, 25)


In [59]:
# Testing contrasts
contrasts = {'Crit':np.array([[0.5,0]]),
            'Neut':np.array([[0,0.5]]),
            'CritNeut':np.array([[0.5,-0.5]])}


print("Computing contrasts...")
for index, (contrast_id, contrast_val) in enumerate(contrasts.items()):
    print(f"  Contrast {index + 1:02g} out of {len(contrasts)}: {contrast_id}")
    # Estimate the contasts.
    z_map = fmri_glm.compute_contrast(contrast_val, output_type="z_score")

    # Write the resulting stat images to file.
    z_image_path = output_dir / f"{contrast_id}_z_map.nii.gz"
    z_map.to_filename(z_image_path)
    


Computing contrasts...
  Contrast 01 out of 3: Crit


<ipython-input-59-b5b93df7d9cd>:11: UserWarning: One contrast given, assuming it for all 2 runs
  z_map = fmri_glm.compute_contrast(contrast_val, output_type="z_score")


  Contrast 02 out of 3: Neut
  Contrast 03 out of 3: CritNeut


In [ ]:
# Testing reporting
report = fmri_glm.generate_report(
    contrasts,
    bg_img=t1,
    title="two-runs fMRI model fitting",
)
statistics = fst_lvl.compute_contrast(contrasts['CritNeut'],
                                      output_type="all")
plotting.plot_stat_map(statistics['z_score'],
                       bg_img=t1,
                       threshold=3,
                       draw_cross=False)
report  # This report can be viewed in a notebook
# report.open_in_browser()

# or we can save as an html file
# from pathlib import Path
output_dir = Path.cwd() / "results" / "plot_two_runs"
output_dir.mkdir(exist_ok=True, parents=True)
report.save_as_html(output_dir / 'report.html')

C:\Users\skarkosz\Anaconda3\lib\site-packages\nilearn\reporting\glm_reporter.py:384: UserWarning: Contrasts will be padded with 17 column(s) of zeros.
  contrast_plot = plot_contrast_matrix(
C:\Users\skarkosz\Anaconda3\lib\site-packages\nilearn\reporting\glm_reporter.py:384: UserWarning: Contrasts will be padded with 23 column(s) of zeros.
  contrast_plot = plot_contrast_matrix(
C:\Users\skarkosz\Anaconda3\lib\site-packages\nilearn\reporting\glm_reporter.py:541: UserWarning: One contrast given, assuming it for all 2 runs
  contrast_id: model.compute_contrast(
C:\Users\skarkosz\Anaconda3\lib\site-packages\nilearn\glm\contrasts.py:108: UserWarning: t contrasts should be of length P=19, but it has length 2. The rest of the contrast was padded with zeros.
  reg = regression_result[label_].Tcontrast(con_val)
C:\Users\skarkosz\Anaconda3\lib\site-packages\nilearn\glm\contrasts.py:108: UserWarning: t contrasts should be of length P=25, but it has length 2. The rest of the contrast was padded w

In [ ]:
# Preparing flow for each subject
directories_list = [n for n in os.listdir() if n.startswith('sub')]
for subject in directories_list:
    fmri_glm = FirstLevelModel(t_r=2,
                    standardize=False,
                    hrf_model='spm',
                    drift_model='cosine',
                    high_pass=0.01)


    img_run1 = nb.load(f"{subject}/ses-1/func/{subject}_ses-1_task-fmri_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz")
    img_run2 = nb.load(f"{subject}/ses-1/func/{subject}_ses-1_task-fmri_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz")
    fmri_imgs = [img_run1,img_run2]
    t1 = nb.load("sub-B045/anat/sub-B045_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz")

    events_1 = get_events(f"trigger/{subject}/ses-1/{subject}_ses-1_Run1.mat")
    events_2 = get_events(f"trigger/{subject}/ses-1/{subject}_ses-1_Run2.mat")

    fst_lvl = fmri_glm.fit(fmri_imgs,events=[events_1,events_2])
    matrices = fst_lvl.design_matrices_
    print(f"Matrices have shapes: {matrices[0].shape}, {matrices[1].shape}")
    report = fmri_glm.generate_report(
        contrasts,
        bg_img=t1,
        title="two-runs fMRI model fitting",
    )

    output_dir = Path.cwd() / "results" / "plot_two_runs"
    output_dir.mkdir(exist_ok=True, parents=True)
    report.save_as_html(output_dir / f'{subject}_report.html')

    contrasts = {'Crit':np.array([[0.5,0]]),
                'Neut':np.array([[0,0.5]]),
                'CritNeut':np.array([[0.5,-0.5]])}


    print("Computing contrasts...")
    for index, (contrast_id, contrast_val) in enumerate(contrasts.items()):
        print(f"  Contrast {index + 1:02g} out of {len(contrasts)}: {contrast_id}")
        # Estimate the contasts.
        z_map = fmri_glm.compute_contrast(contrast_val, output_type="z_score")

        # Write the resulting stat images to file.
        z_image_path = output_dir / f"{contrast_id}_{subject}_z_map.nii.gz"
        z_map.to_filename(z_image_path)

# Second-level analysis

In [ ]:
from nilearn.glm.second_level import make_second_level_design_matrix, SecondLevelModel
design_matrix2nd = make_second_level_design_matrix(directories_list)
def get_1stlvl(subject_list,contrst_id='crit'):
    res = []
    for subject in subject_list:
        res.append(nb.load(f"results/plot_two_runs/{contrast_id}_{subject}_z_map.nii.gz"))
    return res

snd_lvl = SecondLevelModel()
snd_lvl.fit(get_1stlvl(directories_list,'crit'),
            design_matrix=design_matrix2nd)


# Testing MVPA

In [ ]:
directories_list
def get_images(subject_list):
    for subject in subject_list:
        for run in [1,2]:
            yield nb.load(f"{subject}/ses-1/func/{subject}_ses-1_task-fmri_run-0{run}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz")
        
def get_events_generator(subject_list):
    for subject in subject_list:
        for run in [1,2]:
            yield get_events(f"trigger/{subject}/ses-1/{subject}_ses-1_Run{run}.mat")

In [ ]:
imgs= get_images(directories_list)
evts= get_events_generator(directories_list)

In [ ]:
from nilearn.decoding import Decoder
decoder = Decoder(estimator='svc')

In [51]:
# dec1 = decoder.fit(imgs,evts)